In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import plotly.express as py
import plotly.graph_objects as go
import plotly.offline as offline


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/pakistans-largest-ecommerce-dataset/Pakistan Largest Ecommerce Dataset.csv")

In [ ]:
df

# **Dropping empty columns**

In [ ]:
#Dropping the empty columns

df.drop(columns=['Unnamed: 21','Unnamed: 22','Unnamed: 23','Unnamed: 24','Unnamed: 25'],inplace=True)

In [ ]:
df.isnull().sum()

# **Seperating relevant columns as df1**

In [ ]:
df1=df[['payment_method','status','Month','Year','price','qty_ordered']]

In [ ]:
df1.isnull().sum()

In [ ]:
df1.dropna(inplace=True)

In [ ]:
df1.isna().sum()

In [ ]:
df1.columns

In [ ]:
df1['status'].unique()

# **Replacing \\N values with NA**

In [ ]:
df1['status'].replace(['\\N'],'NA',inplace=True)

In [ ]:
df2=df1.pivot_table(index=['status','payment_method'],values=['qty_ordered'],aggfunc=np.sum)

In [ ]:
df1[df1.status=='cod'].shape

In [ ]:
df2

In [ ]:
df2.reset_index(inplace=True)

In [ ]:

offline.init_notebook_mode()


offline.iplot(
    [go.Bar(x=[df2['status'],
               df2['payment_method']],
            y=df2['qty_ordered'])
    ]
)

# **AFTER SEEING THE ABOVE PLOT WE SEE THAT ALOT OF CATEGORIES ARE PRACTICALLY EMPTY OR HAVE VERY LOW NUMBER OF VALUES, removing value: Closed, Fraud, Holded, Paid, NA. We can look at the data without these status types**

In [ ]:
df1.status.unique()

In [ ]:
df3=df1[(df1['status']!= "closed") & 
        (df1['status']!= "fraud") &
        (df1['status']!= "holded") &
        (df1['status']!= "paid")&
        (df1['status']!= "NA")
       ]

In [ ]:
df3.status.unique()

In [ ]:
df4=df3.pivot_table(index=['status','payment_method'],values=['qty_ordered'],aggfunc=np.sum)

In [ ]:
df4.reset_index(inplace=True)

In [ ]:
offline.init_notebook_mode()


offline.iplot(
    [go.Bar(x=[df4['status'],
               df4['payment_method']],
            y=df4['qty_ordered'])
    ]
)

# **In the most worrying status category "canceled" the worst modes of payments are Easy Pay + Easy Pay_MA and Pay Axis** ( I did not merge Easypay and Easypay_MA but it can be done using conditional replacement).

# **Furthermore when we look at successfully completed orders we see that COD or Cash of Delivery is sill the king**. It would be interesting to see if these trends still prevail or are different if we plot Price instead of Order Quantity

In [ ]:
df5=df3.pivot_table(index=['status','payment_method'],values=['price'],aggfunc=np.sum)

In [ ]:
df5.reset_index(inplace=True)

In [ ]:
offline.init_notebook_mode()


offline.iplot(
    [go.Bar(x=[df5['status'],
               df5['payment_method']],
            y=df5['price'])
    ]
)


# **WOW is the only word that comes to my mind. We see a very different trend when we look at the Price variable instead of the quantity variable. In complete status we see that instead of Cash on delivery (COD) , easy pay voucher is the most utilized method of payment when it comes to real Ruppe value. 

# **Furthermore moving to the canceled status we see that Payaxis outstrips Easypay + Easy pay_MA by atleat 76.56M Ruppes when it comes to cancelled orders by Price (Ruppes spent)**

# **This is a very good indicator of the fact that Electronic Payments are perhaps ready to hit main stream in Pakistan but this is also a major Pain Point for online sellers. The combined value of orders being cancelled and which used an electronic mode of payment are outstripping the value being earned through delivered orders. It clearly shows that platforms are clearly struggling with orders that are being paid through e-methods. This could be due to a number of reasons including fradulent orders or lack of fullfillment where the customer is not happy about the product that was delivered


# The graph below (Payment Method Vs Price) shows that the ecommerce industry between 2016,17 and 18 apparently handled more cases of Cancelled Orders and lost more revenue in terms of Cancelled Orders ( Rs 2 Billion) in comparison to actually completed orders. As an ecommerce operator i would be extremely worried about this as it seems that a) I am spending more time, energy and effort on managing the mess being caused by the cancelled orders b) I am loosing revenue opportunity which is approximately worth Rs. 2 Billion **

# Furthermore i think we can safely deduce here that the average order size (based on price) is considerably larger when it comes to Electronics Payment Methods in companrison to COD (Cancelled and Complete both) which constitutes of a much greater volume (Quantity) but looses out when it comes to the actual Ruppee amount.

# **It would also be interesting to see if these trends hold across time between 2017, 17 and 18.**

In [ ]:
df6=df3.pivot_table(index=['status'],values=['price'],aggfunc=np.sum)

In [ ]:
df6

In [ ]:
figure , ax = plt.subplots(figsize=(20,10))

ax.bar(df6.index,df6.price)
ax.set_title('Payment Method Vs Price', fontdict={'fontsize':30})
ax.set_xlabel("Payment Methods",fontdict={'fontsize':15})
ax.set_ylabel("Price",fontdict={'fontsize':15})


plt.xticks(rotation='vertical')

plt.show


In [ ]:
figure , ax = plt.subplots(figsize=(20,10))

ax.bar(df4['payment_method'],df4['qty_ordered'])
ax.set_title('Payment Method Vs Quantity Ordered', fontdict={'fontsize':30})
ax.set_xlabel("Payment Methods",fontdict={'fontsize':15})
ax.set_ylabel("Quantity Ordered",fontdict={'fontsize':15})


plt.xticks(rotation='vertical')

plt.show
